In [95]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix


In [96]:
pwd = os.getcwd()
df = pd.read_excel('/content/Liberty_Sales_2022.xls', sheet_name="RawData")

In [97]:
df.head()


,Customer,Bill Date,Part #,Bill Qty,Cost,Price
0,LI - MIDLAND,2022-12-31,300-4009:STM,6,51.18,54.00
1,LI - MIDLAND,2022-12-31,FREIGHT,1,0.00,35.00
2,LI - MIDLAND,2022-12-31,4918991:CE,1,314.40,355.00
3,LI - WIRELINE YA,2022-12-31,CB313SP:PB,2,48.30,48.18
4,LI - MIDLAND,2022-12-31,632495001031:MYS,6,797.58,1140.00


In [98]:
df.tail()

,Customer,Bill Date,Part #,Bill Qty,Cost,Price
5327,LI - MIDLAND,2022-01-03,BT237JAB:BF,7,42.70,45.36
5328,LI - MIDLAND,2022-01-03,RS3971JAB:BF,7,177.38,188.58
5329,LI - MIDLAND,2022-01-03,P573354:DF,18,812.88,466.92
5330,LI - MIDLAND,2022-01-03,BT287-3JAB:BF,3,42.06,44.73
5331,LI - MIDLAND,2022-01-03,FREIGHT,1,0.00,85.00


In [99]:
df


,Customer,Bill Date,Part #,Bill Qty,Cost,Price
0,LI - MIDLAND,2022-12-31,300-4009:STM,6,51.18,54.00
1,LI - MIDLAND,2022-12-31,FREIGHT,1,0.00,35.00
2,LI - MIDLAND,2022-12-31,4918991:CE,1,314.40,355.00
3,LI - WIRELINE YA,2022-12-31,CB313SP:PB,2,48.30,48.18
4,LI - MIDLAND,2022-12-31,632495001031:MYS,6,797.58,1140.00
...,...,...,...,...,...,...
5327,LI- MIDLAND,2022-01-03,BT237JAB:BF,7,42.70,45.36
5328,LI - MIDLAND,2022-01-03,RS3971JAB:BF,7,177.38,188.58
5329,LI - MIDLAND,2022-01-03,P573354:DF,18,812.88,466.92
5330,LI - MIDLAND,2022-01-03,BT287-3JAB:BF,3,42.06,44.73


Copying the data as a backup

In [100]:
df.columns
datafile = df.copy()
datafile.columns

Index(['Customer', 'Bill Date', 'Part #', 'Bill Qty', 'Cost', 'Price'], dtype='object')

In [101]:
desired_text = ':DF'
filtered_data = df[df['Part #'].str.contains(desired_text, case=False, na=False)]

In [18]:
filtered_data

,Customer,Bill Date,Part #,Bill Qty,Cost,Price
11,LI - ODESSA,2022-12-29,P551316:DF,72,2833.20,1681.92
12,LI - ODESSA,2022-12-29,P781102:DF,68,4042.60,2399.72
13,LI - ODESSA,2022-12-29,P781098:DF,59,5344.22,3172.43
14,LI - ODESSA,2022-12-29,P643216:DF,56,14396.48,8546.16
15,LI - ODESSA,2022-12-29,P636759:DF,40,4886.80,2901.20
...,...,...,...,...,...,...
5297,LI - CIBOLO,2022-01-04,P537877:DF,1,30.22,17.36
5298,LI - CIBOLO,2022-01-04,P537876:DF,1,50.95,29.27
5299,LI - CIBOLO,2022-01-04,P550637:DF,2,24.40,14.02
5300,LI - CIBOLO,2022-01-04,P555776:DF,5,190.90,109.70


In [50]:
# Separating the Customer column into locations
filtered_data.loc[:, 'Customer Type'] = filtered_data['Customer'].str.split('-').str[-1]


<ipython-input-50-e7d8ca83f734>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.loc[:, 'Customer Type'] = filtered_data['Customer'].str.split('-').str[-1]


In [51]:
# Separating the Customer column into locations
filtered_data = filtered_data.copy()
filtered_data['Customer Type'] = filtered_data['Customer'].str.split('-').str[-1]

In [61]:
# Creating separate DataFrames for each customer type
odessa_data = filtered_data[filtered_data['Customer Type'] == ' ODESSA'].drop('Customer Type', axis=1)
midland_data = filtered_data[filtered_data['Customer Type'] == ' MIDLAND'].drop('Customer Type', axis=1)
cibolo_data = filtered_data[filtered_data['Customer Type'] == ' CIBOLO'].drop('Customer Type', axis=1)
gainesville_data = filtered_data[filtered_data['Customer Type'] == ' GAINESVILLE'].drop('Customer Type', axis=1)
williston_data = filtered_data[filtered_data['Customer Type'] == ' WILLISTON'].drop('Customer Type', axis=1)
wireline_data = filtered_data[filtered_data['Customer Type'] == ' WIRELINE YA'].drop('Customer Type', axis=1)
henderson_data = filtered_data[filtered_data['Customer Type'] == ' HENDERSON'].drop('Customer Type', axis=1)
shreveport_data = filtered_data[filtered_data['Customer Type'] == ' SHREVEPORT'].drop('Customer Type', axis=1)


In [62]:
# Adding suffixes to each DataFrame separately
odessa_data = odessa_data.add_suffix(' Odessa')
midland_data = midland_data.add_suffix(' Midland')
cibolo_data = cibolo_data.add_suffix(' Cibolo')
gainesville_data = gainesville_data.add_suffix(' Gainesville')
williston_data = williston_data.add_suffix(' Williston')
wireline_data = wireline_data.add_suffix(' Wireline')
henderson_data = henderson_data.add_suffix(' Henderson')
shreveport_data = shreveport_data.add_suffix(' Shreveport')

In [63]:
print(odessa_data.columns)
print(midland_data.columns)



Index(['Customer Odessa', 'Bill Date Odessa', 'Part # Odessa',
       'Bill Qty Odessa', 'Cost Odessa', 'Price Odessa'],
      dtype='object')
Index(['Customer Midland', 'Bill Date Midland', 'Part # Midland',
       'Bill Qty Midland', 'Cost Midland', 'Price Midland'],
      dtype='object')


In [76]:

merged_data = odessa_data
merged_data = midland_data
merged_data = cibolo_data
merged_data = gainesville_data
merged_data = williston_data
merged_data = wireline_data
merged_data = henderson_data
merged_data = shreveport_data

In [77]:
dataframes_to_merge = [midland_data, cibolo_data, gainesville_data, williston_data, wireline_data, henderson_data, shreveport_data]

In [78]:
merge_columns = ['Customer', 'Bill Date', 'Part #', 'Bill Qty', 'Cost', 'Price']

In [79]:
print(merge_columns)


['Customer', 'Bill Date', 'Part #', 'Bill Qty', 'Cost', 'Price']


In [86]:
output.to_excel(pwd + '/Final_Ouput2.xlsx')

In [108]:
merged_data


,Customer Odessa,Bill Date Odessa,Part # Odessa,Bill Qty Odessa,Cost Odessa,Price Odessa,Customer Midland,Bill Date Midland,Part # Midland,Bill Qty Midland,...,Part # Henderson,Bill Qty Henderson,Cost Henderson,Price Henderson,Customer Shreveport,Bill Date Shreveport,Part # Shreveport,Bill Qty Shreveport,Cost Shreveport,Price Shreveport
11,LI - ODESSA,2022-12-29,P551316:DF,72.0,2833.20,1681.92,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
12,LI - ODESSA,2022-12-29,P781102:DF,68.0,4042.60,2399.72,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
13,LIBERTY OILFIELD - ODESSA,2022-12-29,P781098:DF,59.0,5344.22,3172.43,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
14,LI - ODESSA,2022-12-29,P643216:DF,56.0,14396.48,8546.16,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
15,LI - ODESSA,2022-12-29,P636759:DF,40.0,4886.80,2901.20,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5168,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3634,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,P552020:DF,36.0,407.52,241.92,NaN,NaT,NaN,NaN,NaN,NaN
3635,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,DBF8727:DF,20.0,1616.80,959.80,NaN,NaT,NaN,NaN,NaN,NaN


In [109]:
filtered_data


,Customer,Bill Date,Part #,Bill Qty,Cost,Price
11,LI - ODESSA,2022-12-29,P551316:DF,72,2833.20,1681.92
12,LI - ODESSA,2022-12-29,P781102:DF,68,4042.60,2399.72
13,LI - ODESSA,2022-12-29,P781098:DF,59,5344.22,3172.43
14,LI - ODESSA,2022-12-29,P643216:DF,56,14396.48,8546.16
15,LI - ODESSA,2022-12-29,P636759:DF,40,4886.80,2901.20
...,...,...,...,...,...,...
5297,LI - CIBOLO,2022-01-04,P537877:DF,1,30.22,17.36
5298,LI - CIBOLO,2022-01-04,P537876:DF,1,50.95,29.27
5299,LI - CIBOLO,2022-01-04,P550637:DF,2,24.40,14.02
5300,LI - CIBOLO,2022-01-04,P555776:DF,5,190.90,109.70


In [110]:
dataframes_to_merge

[                Customer Midland Bill Date Midland Part # Midland  \
 25    LI- MIDLAND        2022-12-28     P636759:DF   
 27    LI - MIDLAND        2022-12-28     P822768:DF   
 32    LI - MIDLAND        2022-12-28     DBL7405:DF   
 48    LI - MIDLAND        2022-12-27     P551010:DF   
 49    LI - MIDLAND        2022-12-27     P532510:DF   
 ...                          ...               ...            ...   
 5230  LI - MIDLAND        2022-01-07     P537876:DF   
 5241  LI - MIDLAND        2022-01-06     P573354:DF   
 5279  LI - MIDLAND        2022-01-05     P165659:DF   
 5294  LI - MIDLAND        2022-01-05     P550348:DF   
 5329  LI - MIDLAND        2022-01-03     P573354:DF   
 
       Bill Qty Midland  Cost Midland  Price Midland  
 25                  12       1466.04         870.36  
 27                   8        207.20         123.04  
 32                   1         30.59          18.16  
 48                  10        284.90         169.10  
 49                   3 